# Refinitiv Data Library for Python

## Delivery layer - RDP Stream - IPA FX Cross
This notebook demonstrates how to use an RDP stream to subscribe to an FX Cross Intrument Pricing Analytic.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.delivery import rdp_stream
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fe6f220cb90 {name='workspace'}>

## Retrieve data

### Define a calback to display received events

In [4]:
def display_event(stream, eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(event)

### Create an RDP stream and register event callbacks

In [5]:
stream = rdp_stream.Definition(
    service = None,
    api = "streaming/quantitative-analytics/financial-contracts",
    universe = {
        "instrumentType": "FxCross", 
        "instrumentDefinition": {
            "instrumentTag": "USDAUD", 
            "fxCrossType": "FxSpot", 
            "fxCrossCode": "USDAUD"
        }
    }, 
    parameters = None,
    extended_params = {
        "marketData": {
            "fxSpots": [
                {
                    "spotDefinition": {
                        "fxCrossCode": "AUDUSD", 
                        "Source": "Composite"
                    }
                }
            ]
        }
    },
    view = ["InstrumentTag", "FxSpot_BidMidAsk", "ErrorCode", "Ccy1SpotDate", "Ccy2SpotDate"],
).get_stream()

In [6]:
stream.on_response(lambda stream, event: display_event(stream, 'response', event))
stream.on_update(lambda stream, event: display_event(stream, 'update', event))
stream.on_alarm(lambda stream, event: display_event(stream, 'alarm', event))

### Open the stream
This example uses the open() synchronous call to open the stream. Because it is a synchronous call, the first notification (either via on_refresh(), on_status() or on_error()) happens before the open() method returns. If the open_async() asynchronous method is used instead, the first notification happens after open_async() returns.

In [7]:
stream.open()

----------------------------------------------------------
>>> response event received at 17:02:57.933885
{'data': [['USDAUD', {'bid': 1.3906271728549577, 'ask': 1.3910140492418974, 'mid': 1.3908206110484276, 'processingInformation': ''}, '', '2021-12-20', '2021-12-20']], 'type': 'Response', 'streamID': '1'}


<OpenState.Opened: 1>

----------------------------------------------------------
>>> update event received at 17:03:00.914925
{'data': [['USDAUD', {'bid': 1.3906271728549577, 'ask': 1.3910140492418974, 'mid': 1.3908206110484276, 'processingInformation': ''}, '', '2021-12-20', '2021-12-20']], 'type': 'Update', 'streamID': '1'}
----------------------------------------------------------
>>> update event received at 17:03:01.913708
{'data': [['USDAUD', {'bid': 1.3906271728549577, 'ask': 1.3910140492418974, 'mid': 1.3908206110484276, 'processingInformation': ''}, '', '2021-12-20', '2021-12-20']], 'type': 'Update', 'streamID': '1'}
----------------------------------------------------------
>>> update event received at 17:03:03.933867
{'data': [['USDAUD', {'bid': 1.3906271728549577, 'ask': 1.3910140492418974, 'mid': 1.3908206110484276, 'processingInformation': ''}, '', '2021-12-20', '2021-12-20']], 'type': 'Update', 'streamID': '1'}
----------------------------------------------------------
>>> update event receiv

### Close the stream

In [8]:
stream.close()

----------------------------------------------------------
>>> update event received at 17:03:05.909605
{'data': [['USDAUD', {'bid': 1.3906271728549577, 'ask': 1.3910140492418974, 'mid': 1.3908206110484276, 'processingInformation': ''}, '', '2021-12-20', '2021-12-20']], 'type': 'Update', 'streamID': '1'}


<OpenState.Closed: 3>

## Close the session

In [9]:
rd.close_session()